<a href="https://colab.research.google.com/github/girishpalange/DS_Projects/blob/main/Kaggle_DisaterTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd


In [ ]:
train = pd.read_csv('train.csv')

test = pd.read_csv('test.csv')

In [ ]:
#train.info()
#train['text'].isna().values.any()
test['text'].head(10)

0                   Just happened a terrible car crash
1    Heard about #earthquake is different cities, s...
2    there is a forest fire at spot pond, geese are...
3             Apocalypse lighting. #Spokane #wildfires
4        Typhoon Soudelor kills 28 in China and Taiwan
5                   We're shaking...It's an earthquake
6    They'd probably still show more life than Arse...
7                                    Hey! How are you?
8                                     What a nice hat?
9                                            Fuck off!
Name: text, dtype: object

In [ ]:
train_sentences = []
train_labels = []

for s in train['text']:
  train_sentences.append(s)

for l in train['target']:
  train_labels.append(l)  

 

In [ ]:
train_sentences = np.array(train_sentences)
train_labels = np.array(train_labels)



In [ ]:
from sklearn.model_selection import train_test_split

train_sentences, test_sentences, train_labels, test_labels = train_test_split(train_sentences, train_labels, test_size=0.1)

In [ ]:
vocab_size = 10000
embedding_dim = 32
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)
testing_sequences = tokenizer.texts_to_sequences(test_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [ ]:
model = tf.keras.Sequential([
tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(6, activation='relu'),
tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 120, 32)           320000    
                                                                 
 flatten_6 (Flatten)         (None, 3840)              0         
                                                                 
 dense_12 (Dense)            (None, 6)                 23046     
                                                                 
 dense_13 (Dense)            (None, 1)                 7         
                                                                 
Total params: 343,053
Trainable params: 343,053
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 10
model.fit(padded,
train_labels,
epochs=num_epochs,
validation_data=(testing_padded, test_labels))

Epoch 1/10
172/172 [==============================] - 2s 8ms/step - loss: 0.6279 - accuracy: 0.6504 - val_loss: 0.5550 - val_accuracy: 0.6995
Epoch 2/10
172/172 [==============================] - 1s 8ms/step - loss: 0.4319 - accuracy: 0.8181 - val_loss: 0.4513 - val_accuracy: 0.7997
Epoch 3/10
172/172 [==============================] - 1s 8ms/step - loss: 0.2675 - accuracy: 0.9004 - val_loss: 0.4493 - val_accuracy: 0.8194
Epoch 4/10
172/172 [==============================] - 1s 7ms/step - loss: 0.1590 - accuracy: 0.9475 - val_loss: 0.4852 - val_accuracy: 0.8161
Epoch 5/10
172/172 [==============================] - 1s 7ms/step - loss: 0.0983 - accuracy: 0.9686 - val_loss: 0.5183 - val_accuracy: 0.7997
Epoch 6/10
172/172 [==============================] - 1s 7ms/step - loss: 0.0684 - accuracy: 0.9779 - val_loss: 0.5453 - val_accuracy: 0.7833
Epoch 7/10
172/172 [==============================] - 1s 8ms/step - loss: 0.0546 - accuracy: 0.9808 - val_loss: 0.5787 - val_accuracy: 0.7980
Epoch 

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test['text'])
test_padded = pad_sequences(test_sequences,maxlen=max_length)

In [ ]:
pred = []

pred = model.predict(test_padded)

pred1 = [1 if x >= 0.5 else 0 for x in pred]

102/102 [==============================] - 0s 2ms/step


In [ ]:
test['target'] = pred1
test.head(10)

,id,keyword,location,text,target
0,0,NaN,NaN,Just happened a terrible car crash,1
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,0
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1
5,12,NaN,NaN,We're shaking...It's an earthquake,1
6,21,NaN,NaN,They'd probably still show more life than Arse...,0
7,22,NaN,NaN,Hey! How are you?,0
8,27,NaN,NaN,What a nice hat?,0
9,29,NaN,NaN,Fuck off!,0


In [ ]:
test.to_csv('submission.csv',columns=['id','target'],index=False)